## Customer Support Ticket Classification Test 2 (Ok)
### Pros
* Some Structured data via pydantic models
* Confident Score
### Cons
* We don't give the AI context, Why are there ticket? Who are these tickets for etc. 
* LLM Model have a harder time to answer in a good way without context.

In [1]:
from pydantic_model import *
from tickets import *
from ollama_connect import *
from IPython.display import display
import pandas as pd

MODELNAME = "llama3.1:8b"
rows = []

In [2]:
def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client_instructor.chat.completions.create(
        model=MODELNAME,
        response_model=TicketClassification,
        temperature=0,
        max_tokens=16384,
        seed=633242,
        top_p=0.2,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": "Analyze the following customer support ticket and extract the requested information."
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

In [ ]:
for ticket in tickets:
    result = classify_ticket(ticket)
    data = result.model_dump()
    # Ensure all possibly None fields are set to a default value for DataFrame compatibility
    if data.get("suggested_action") is None:
        data["suggested_action"] = ""
    if data.get("key_information") is None:
        data["key_information"] = []
    if data.get("confidence") is None:
        data["confidence"] = 0.0
    if data.get("contain_PII_info") is None:
        data["contain_PII_info"] = 0.0
    if data.get("pii_type") is None:
        data["pii_type"] = "NONE"
    data['raw_ticket'] = ticket.strip()
    rows.append(data)
df = pd.DataFrame(rows)
